## 1. Test Baseline Before Finetuning
### 1.1 load and process ade dataset

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset
import random
dataset = load_dataset("ade-benchmark-corpus/ade_corpus_v2", "Ade_corpus_v2_drug_ade_relation")

# 查看数据集内容
print(dataset)
print(dataset['train'][8])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/491k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/6821 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'drug', 'effect', 'indexes'],
        num_rows: 6821
    })
})
{'text': 'RESULTS: A 44-year-old man taking naproxen for chronic low back pain and a 20-year-old woman on oxaprozin for rheumatoid arthritis presented with tense bullae and cutaneous fragility on the face and the back of the hands.', 'drug': 'naproxen', 'effect': 'tense bullae', 'indexes': {'drug': {'start_char': [34], 'end_char': [42]}, 'effect': {'start_char': [146], 'end_char': [158]}}}


In [ ]:
def create_prompt_components(icl_examples):
    """
    创建 schema-aware prompt 和 ICL prompt，并返回二者。
    """

    # 创建 schema-aware prompt
    schema_prompt = """
    from typing import List
    class Rel:
        def __init__(self, name: str):
            self.name = name

    class Cause(Rel):
        def __init__(self, name: str):
            super().__init__(name=name)

    class Entity:
        def __init__(self, name: str):
            self.name = name

    class drug(Entity):
        def __init__(self, name: str):
            super().__init__(name=name)

    class adverse_effect(Entity):
        def __init__(self, name: str):
            super().__init__(name=name)

    class Triple:
        def __init__(self, head: Entity, relation: Rel, tail: Entity):
            self.head = head
            self.relation = relation
            self.tail = tail

    class Extract:
        def __init__(self, triples: List[Triple] = []):
            self.triples = triples
    """

    # 生成 ICL prompt：分三步进行推理
    icl_prompt = ""
    for i in range(len(icl_examples['text'])):
        drug = icl_examples['drug'][i]
        effect = icl_examples['effect'][i]
        text = icl_examples['text'][i]

        # # Step 1: 关系识别
        # icl_prompt += f'""" {text} """\n'
        # icl_prompt += "# Step 1: 识别所有可能的关系\n"
        # icl_prompt += f"# Rel('Adverse Effect')\n\n"

        # # Step 2: 实体提取
        # icl_prompt += "# Step 2: 提取与关系相关的实体\n"
        # icl_prompt += f"# drug('{drug}'), adverse_effect('{effect}')\n\n"

        # Step 3: 生成最终的三元组
        #icl_prompt += "# Step 3: 生成最终的三元组\n"
        icl_prompt += "#生成最终的三元组:\n"
        icl_prompt += f"extract = Extract([Triple(drug('{drug}'), Cause('Adverse Effect'), adverse_effect('{effect}'))])\n\n"

    return schema_prompt, icl_prompt


def generate_prompt(text, schema_prompt, icl_prompt):
    """
    生成 CodeT5 模型的输入 prompt，不包含 label。

    Args:
    - text (str): 自然语言文本
    - schema_prompt (str): schema-aware prompt (结构化提示)
    - icl_prompt (str): In-context learning 示例 prompt

    Returns:
    - str: 最终生成的 prompt
    """
    # 将自然语言文本包裹到适当的格式中
    test_text = '\n""" ' + text + ' """'
    # 拼接 schema prompt、ICL prompt 和当前测试文本
    prompt = schema_prompt + '\n' + icl_prompt + test_text
    return prompt

def prepare_input_data(text, schema_prompt, icl_prompt, true_label):

    # 生成最终的 prompt
    prompt = generate_prompt(text, schema_prompt, icl_prompt)

    # 返回合并后的输入数据
    return {
        'text':text,
        'prompt': prompt,
        'true_label': true_label
    }

def select_icl_examples(dataset, num_examples=3):
    selected_examples = {
        'text': [],
        'drug': [],
        'effect': [],
        'indexes': []
    }

    indices = random.sample(range(len(dataset)), num_examples)
    for idx in indices:
        sample = dataset[idx]
        selected_examples['text'].append(sample['text'])
        selected_examples['drug'].append(sample['drug'])
        selected_examples['effect'].append(sample['effect'])
        selected_examples['indexes'].append(idx)

    return selected_examples

def create_and_output_test_data(dataset, icl_dataset, num_samples=None):
    num_samples = num_samples if num_samples is not None else len(dataset)
    processed_data = []

    for i in range(num_samples):
        sample = dataset[i]
        text = sample['text']
        drug = sample['drug']
        effect = sample['effect']
        true_label = f'Extract([Triple(drug("{drug}"), Cause("Adverse Effect"), adverse_effect("{effect}"))])'

        # Randomly select ICL examples for each sample
        icl_examples = select_icl_examples(icl_dataset, 2)
        schema_prompt, icl_prompt = create_prompt_components(icl_examples)

        # Prepare input data
        input_data = prepare_input_data(text, schema_prompt, icl_prompt, true_label)
        processed_data.append(input_data)

    return processed_data


processed_test_data = create_and_output_test_data(dataset['train'], icl_dataset=dataset['train'], num_samples=10)

In [ ]:
print(processed_test_data[0]['prompt'])


    from typing import List
    class Rel:
        def __init__(self, name: str):
            self.name = name

    class Cause(Rel):
        def __init__(self, name: str):
            super().__init__(name=name)

    class Entity:
        def __init__(self, name: str):
            self.name = name

    class drug(Entity):
        def __init__(self, name: str):
            super().__init__(name=name)

    class adverse_effect(Entity):
        def __init__(self, name: str):
            super().__init__(name=name)

    class Triple:
        def __init__(self, head: Entity, relation: Rel, tail: Entity):
            self.head = head
            self.relation = relation
            self.tail = tail

    class Extract:
        def __init__(self, triples: List[Triple] = []):
            self.triples = triples
    
#生成最终的三元组:
extract = Extract([Triple(drug('itraconazole'), Cause('Adverse Effect'), adverse_effect('acute hepatitis'))])

#生成最终的三元组:
extract = Extract([Triple(drug('IFN-alpha'), Ca

### 1.2 Evaluation method of ade extraction

In [ ]:
import re

def parse_generated_code(generated_code):
    """
    从生成的代码中提取三元组 (drug, relation, adverse_effect)。
    """
    triples = []
    # 使用正则表达式匹配 Triple(drug("name"), Cause("relation"), adverse_effect("effect")) 的结构
    pattern = r"Triple\s*\(\s*drug\s*\(\s*(?:'|\")([^'\"]+)(?:'|\")\s*\)\s*,\s*Cause\s*\(\s*(?:'|\")([^'\"]+)(?:'|\")\s*\)\s*,\s*adverse_effect\s*\(\s*(?:'|\")([^'\"]+)(?:'|\")\s*\)\s*\)"

    matches = re.findall(pattern, generated_code)

    # 打印匹配到的内容
    print(f"Matched Triples: {matches}")

    for match in matches:
        drug, relation, effect = match
        triples.append((drug.strip(), relation.strip(), effect.strip()))

    return triples

def evaluate_entity_accuracy(generated_triples, true_triples):
    """
    比较生成的 (drug, relation, adverse_effect) 中的实体，并计算药物和不良反应的准确率。

    Args:
    - generated_triples (list): 生成的 (drug, relation, adverse_effect) 三元组列表。
    - true_triples (list): 真实的 (drug, relation, adverse_effect) 三元组列表。

    Returns:
    - dict: 药物和不良反应的准确率。
    """
    # 提取生成的药物和不良反应实体集合
    generated_drugs = set([gen_drug for gen_drug, _, _ in generated_triples])
    generated_effects = set([gen_effect for _, _, gen_effect in generated_triples])

    # 提取真实的药物和不良反应实体集合
    true_drugs = set([true_drug for true_drug, _, _ in true_triples])
    true_effects = set([true_effect for _, _, true_effect in true_triples])

    # 计算正确匹配的药物和不良反应实体数量
    correct_drugs = len(generated_drugs & true_drugs)
    correct_effects = len(generated_effects & true_effects)

    # 总的实体数量
    total_drugs = len(true_drugs)
    total_effects = len(true_effects)

    # 计算药物和不良反应的准确率
    drug_accuracy = correct_drugs / total_drugs if total_drugs > 0 else 0
    effect_accuracy = correct_effects / total_effects if total_effects > 0 else 0

    return {
        'drug_accuracy': drug_accuracy,
        'effect_accuracy': effect_accuracy
    }



### 1.3 load different open-souce model for testing

In [ ]:
#!pip install sacremoses

In [ ]:
from transformers import RobertaTokenizer, RobertaForMaskedLM
from transformers import AutoTokenizer, AutoModelForCausalLM
# from transformers import BioGptTokenizer, BioGptForConditionalGeneration
from transformers import BioGptTokenizer, BioGptForCausalLM
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import RobertaTokenizer, T5ForConditionalGeneration
import torch

tokenizer = RobertaTokenizer.from_pretrained("Salesforce/codet5-base")
model = T5ForConditionalGeneration.from_pretrained("Salesforce/codet5-base")

# tokenizer = AutoTokenizer.from_pretrained("Salesforce/codet5-large")
# model = AutoModelForSeq2SeqLM.from_pretrained("Salesforce/codet5-large")

# tokenizer = AutoTokenizer.from_pretrained("ThoughtFocusAI/CodeGeneration-CodeT5-base")
# model = AutoModelForSeq2SeqLM.from_pretrained("ThoughtFocusAI/CodeGeneration-CodeT5-base",from_tf=True)

# tokenizer = BioGptTokenizer.from_pretrained("microsoft/biogpt")
# model = BioGptForCausalLM.from_pretrained("microsoft/biogpt")

# tokenizer = AutoTokenizer.from_pretrained("Salesforce/codegen-16B-multi")
# model = AutoModelForCausalLM.from_pretrained("Salesforce/codegen-16B-multi")

# Load model directly
# from transformers import AutoModelForSeq2SeqLM
# tokenizer = AutoTokenizer.from_pretrained("Salesforce/codet5p-6b")
# model = AutoModelForSeq2SeqLM.from_pretrained("Salesforce/codet5p-6b", trust_remote_code=True)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/703k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/294k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/12.5k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [ ]:
from transformers import RobertaTokenizer, T5ForConditionalGeneration
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import os
import torch
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
def run_baseline_with_evaluation(processed_test_data, tokenizer,model, max_length=1024 , num_beams=5, temperature=0.7, use_gpu=True):
    device = torch.device('cuda' if torch.cuda.is_available() and use_gpu else 'cpu')
    model = model.to(device)
    model.eval()
    results = []
    total_drug_accuracy, total_effect_accuracy = 0, 0
    num_samples = len(processed_test_data)
    for i, data in enumerate(processed_test_data[3:5]):
        input_prompt = data['prompt']
        true_label = data['true_label']
        try:
          inputs = tokenizer.encode(input_prompt, return_tensors="pt", max_length=max_length, truncation=True).to(device)
        except Exception as e:
          print(f"Error during tokenization: {e}")
        with torch.no_grad():
            outputs = model.generate(
                inputs,
                max_new_tokens=max_length,
                num_beams=num_beams,
                temperature=temperature,
                early_stopping=True
            )
        generated_code = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # 解析生成的代码和真实标签
        generated_triples = parse_generated_code(generated_code)
        true_triples = parse_generated_code(true_label)
        generated_triples = [(drug.lower(), relation.lower(), effect.lower()) for drug, relation, effect in generated_triples]
        true_triples = [(drug.lower(), relation.lower(), effect.lower()) for drug, relation, effect in true_triples]
        # 评估生成的三元组
        evaluation_metrics = evaluate_entity_accuracy(generated_triples, true_triples)

        # 累积计算 drug_accuracy, effect_accuracy
        total_drug_accuracy += evaluation_metrics['drug_accuracy']
        total_effect_accuracy += evaluation_metrics['effect_accuracy']

        # 打印结果
        print(f"Inputed text: {data['text']}\n")
        print(f"Sample {i+1} - Generated Code:\n{generated_code}\n")
        print(f"Sample {i+1} - generated_triples:\n{generated_triples}\n")
        print(f"Sample {i+1} - true_triples:\n{true_triples}\n")
        print(f"Sample {i+1} - True Label:\n{true_label}\n")
        print(f"Evaluation Metrics: {evaluation_metrics}\n")

        results.append({
            'generated_code': generated_code,
            'true_label': true_label,
            'evaluation_metrics': evaluation_metrics
        })

    # 计算平均 drug_accuracy 和 effect_accuracy
    avg_drug_accuracy = total_drug_accuracy / num_samples
    avg_effect_accuracy = total_effect_accuracy / num_samples

    # 打印最终的平均评估结果
    print(f"Average Drug Accuracy: {avg_drug_accuracy:.4f}")
    print(f"Average Effect Accuracy: {avg_effect_accuracy:.4f}")

    # 返回结果
    return {
        'results': results,
        'average_metrics': {
            'drug_accuracy': avg_drug_accuracy,
            'effect_accuracy': avg_effect_accuracy
        }
    }


# 调用 run_baseline 函数来评估模型，并在 GPU 上运行
baseline_results = run_baseline_with_evaluation(processed_test_data, tokenizer,model,use_gpu=True)


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Matched Triples: []
Matched Triples: [('naproxen', 'Adverse Effect', 'pseudoporphyria')]
Inputed text: METHODS: We report two cases of pseudoporphyria caused by naproxen and oxaprozin.

Sample 1 - Generated Code:
(self, triples: List[Triple]=[]):
            self.triples.append(Triple(drug('salazosulfapyridine'), Cause('Adverse Effect'),(self, triples: List[Triple]=[]):
            self.triples.append(Triple(drug('salazosulfapyridine'), Cause('Adverse Effect'),(self, triples: List[Triple]=[]

Sample 1 - generated_triples:
[]

Sample 1 - true_triples:
[('naproxen', 'adverse effect', 'pseudoporphyria')]

Sample 1 - True Label:
Extract([Triple(drug("naproxen"), Cause("Adverse Effect"), adverse_effect("pseudoporphyria"))])

Evaluation Metrics: {'drug_accuracy': 0.0, 'effect_accuracy': 0.0}

Matched Triples: []
Matched Triples: [('oxaprozin', 'Adverse Effect', 'pseudoporphyria')]
Inputed text: METHODS: We report two cases of pseudoporphyria caused by naproxen and oxaprozin.

Sample 2 - Gene

## 2. Finetuning the CodeT5

In [ ]:
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 18.3 MB/s eta 0:00:00


In [ ]:
# 执行数据集的划分
train_test_split = dataset['train'].train_test_split(test_size=0.3, seed=42)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']
n=len(train_dataset)
processed_train_data = create_and_output_test_data(train_dataset, icl_dataset=train_dataset,num_samples=n)
n_test = len(test_dataset)
# Process the test dataset
processed_test_data = create_and_output_test_data(test_dataset,icl_dataset=train_dataset, num_samples=n_test)

In [ ]:
import torch
from torch.utils.data import DataLoader
from transformers import AdamW, get_linear_schedule_with_warmup
from peft import get_peft_model, LoraConfig, TaskType
from tqdm import tqdm
def finetuning_Lora(train_data, test_data, model, tokenizer, epochs=3, batch_size=8, learning_rate=5e-4,
                    max_length=512, num_beams=5, temperature=0.7, use_gpu=True, accumulation_steps=4):

    import torch
    from torch.utils.data import DataLoader
    from transformers import AdamW, get_linear_schedule_with_warmup
    from peft import get_peft_model, LoraConfig, TaskType
    from tqdm import tqdm

    device = torch.device('cuda' if torch.cuda.is_available() and use_gpu else 'cpu')
    model = model.to(device)

    # Configure LoRA
    peft_config = LoraConfig(
        task_type=TaskType.SEQ_2_SEQ_LM,   # For sequence-to-sequence models
        inference_mode=False,
        r=16,                              # Rank of the LoRA matrices
        lora_alpha=32,
        lora_dropout=0.1,
    )

    # Wrap the model with PEFT
    model = get_peft_model(model, peft_config)
    model.print_trainable_parameters()

    # Create DataLoaders
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_data, batch_size=1, shuffle=False)

    # Define optimizer and scheduler
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    total_steps = len(train_loader) * epochs
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=int(0.1 * total_steps),
        num_training_steps=total_steps
    )

    # Training loop
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        optimizer.zero_grad()
        for i, batch in enumerate(tqdm(train_loader, desc="Training")):
            # ... prepare inputs ...
            outputs = model(
                input_ids=batch['input_ids'].to(device),
                attention_mask=batch['attention_mask'].to(device),
                labels=batch['labels'].to(device)
            )
            loss = outputs.loss
            loss = loss / accumulation_steps
            loss.backward()
            if (i + 1) % accumulation_steps == 0:
                optimizer.step()
                optimizer.zero_grad()
            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)
        print(f"Average training loss: {avg_loss:.4f}")

        # Evaluation after each epoch
        model.eval()
        total_drug_accuracy, total_effect_accuracy = 0, 0
        num_samples = len(test_loader)

        with torch.no_grad():
            for batch in tqdm(test_loader, desc="Evaluating"):
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)

                outputs = model.generate(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    max_length=max_length,
                    num_beams=num_beams,
                    temperature=temperature,
                    early_stopping=True
                )
                generated_code = tokenizer.decode(outputs[0], skip_special_tokens=True)
                true_label = tokenizer.decode(labels[0], skip_special_tokens=True)
                print(generated_code)
                print(true_label)
                # Parse generated code and true label
                generated_triples = parse_generated_code(generated_code)
                true_triples = parse_generated_code(true_label)

                # Lowercase for comparison
                generated_triples = [(drug.lower(), relation.lower(), effect.lower()) for drug, relation, effect in generated_triples]
                true_triples = [(drug.lower(), relation.lower(), effect.lower()) for drug, relation, effect in true_triples]

                # Evaluate
                evaluation_metrics = evaluate_entity_accuracy(generated_triples, true_triples)
                total_drug_accuracy += evaluation_metrics['drug_accuracy']
                total_effect_accuracy += evaluation_metrics['effect_accuracy']

        avg_drug_accuracy = total_drug_accuracy / num_samples
        avg_effect_accuracy = total_effect_accuracy / num_samples

        print(f"Evaluation after epoch {epoch+1}:")
        print(f"Average Drug Accuracy: {avg_drug_accuracy:.4f}")
        print(f"Average Effect Accuracy: {avg_effect_accuracy:.4f}")

    # Save the LoRA adapters
    model.save_pretrained('lora_model')

    return model
import torch

class CodeKGC_Dataset(torch.utils.data.Dataset):
    def __init__(self, data_list, tokenizer, max_length=512):

        self.data_list = data_list
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):

        return len(self.data_list)

    def __getitem__(self, idx):

        data = self.data_list[idx]

        # 编码输入文本（Prompt）
        input_encoding = self.tokenizer.encode_plus(
            data['prompt'],
            truncation=True,
            max_length=self.max_length,
            padding='max_length',
            return_tensors='pt'
        )

        # 编码标签（True Label）
        label_encoding = self.tokenizer.encode_plus(
            data['true_label'],
            truncation=True,
            max_length=self.max_length,
            padding='max_length',
            return_tensors='pt'
        )

        # 获取编码后的张量
        input_ids = input_encoding['input_ids'].squeeze()
        attention_mask = input_encoding['attention_mask'].squeeze()
        labels = label_encoding['input_ids'].squeeze()

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels
        }


### 2.1 Training and test without codekgc

In [ ]:
# 加载数据集
from datasets import load_dataset
dataset = load_dataset("ade-benchmark-corpus/ade_corpus_v2", "Ade_corpus_v2_drug_ade_relation")


def generate_prompt(text):
    """
    生成模型的输入 prompt，不包含其他信息。

    Args:
    - text (str): 自然语言文本

    Returns:
    - str: 最终生成的 prompt，仅为文本内容
    """
    # 直接返回文本内容
    prompt = text
    return prompt

def prepare_input_data(text, true_label):
    """
    将 text 和 true_label 合并为一个字典，以供模型训练使用。

    Args:
    - text (str): 自然语言文本
    - true_label (str): 样本的真实标签

    Returns:
    - dict: 包含 'prompt' 和 'true_label' 的字典
    """
    # 生成最终的 prompt（仅为文本）
    prompt = generate_prompt(text)

    # 返回合并后的输入数据
    return {
        'text': text,
        'prompt': prompt,
        'true_label': true_label
    }

def create_and_output_test_data(dataset, num_samples=None):
    num_samples = num_samples if num_samples is not None else len(dataset)
    processed_data = []

    for i in range(num_samples):
        sample = dataset[i]
        text = sample['text']
        drug = sample['drug']
        effect = sample['effect']
        true_label = f'Extract([Triple(drug("{drug}"), Cause("Adverse Effect"), adverse_effect("{effect}"))])'

        # 不再需要 schema_prompt 和 icl_prompt
        # Prepare input data
        input_data = prepare_input_data(text, true_label)
        processed_data.append(input_data)

    return processed_data

# 执行数据集的划分
train_test_split = dataset['train'].train_test_split(test_size=0.3, seed=42)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

# 处理训练数据
processed_train_data = create_and_output_test_data(train_dataset)

# 处理测试数据
processed_test_data = create_and_output_test_data(test_dataset)

# 创建数据集实例
from transformers import T5ForConditionalGeneration, RobertaTokenizer

# 加载预训练模型和分词器
tokenizer = RobertaTokenizer.from_pretrained("Salesforce/codet5-base")
model = T5ForConditionalGeneration.from_pretrained("Salesforce/codet5-base")

# 创建数据集实例
train_data = CodeKGC_Dataset(processed_train_data[10:500], tokenizer)
test_data = CodeKGC_Dataset(processed_test_data[:5], tokenizer)


In [ ]:
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 8.1 MB/s eta 0:00:00


In [ ]:
import torch
from torch.utils.data import DataLoader
from transformers import AdamW, get_linear_schedule_with_warmup
from peft import get_peft_model, LoraConfig, TaskType
from tqdm import tqdm
from transformers import T5ForConditionalGeneration, RobertaTokenizer
def prepare_input_data(text, answer):

    prompt = f"Please extract the drug-adverse effect triples from the following text:\n\n{text}\n\nExtracted triples:"
    return {
        'prompt': prompt,
        'answer': answer.strip()  # Remove leading/trailing whitespace
    }

def create_and_output_test_data(dataset, num_samples=None):
    num_samples = num_samples if num_samples is not None else len(dataset)
    processed_data = []

    for i in range(num_samples):
        sample = dataset[i]
        text = sample['text']
        drug = sample['drug']
        effect = sample['effect']
        answer = f" Extract([Triple(drug('{drug}'), Cause('Adverse Effect'), adverse_effect('{effect}'))])"

        # Prepare input data with the new function
        input_data = prepare_input_data(text, answer)
        processed_data.append(input_data)

    return processed_data
import torch

class CodeKGC_Dataset(torch.utils.data.Dataset):
    def __init__(self, data_list, tokenizer, max_length=256):
        """
        Initializes the dataset.

        Args:
        - data_list (list): List of dictionaries with 'prompt' and 'answer'.
        - tokenizer: The tokenizer to use.
        - max_length (int): Maximum sequence length.
        """
        self.data_list = data_list
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        """
        Returns the number of samples in the dataset.
        """
        return len(self.data_list)

    def __getitem__(self, idx):
        """
        Returns a sample at the given index.

        Args:
        - idx (int): Index of the sample.

        Returns:
        - dict: A dictionary containing 'input_ids', 'attention_mask', and 'labels'.
        """
        data = self.data_list[idx]

        # Encode the input prompt
        input_encoding = self.tokenizer(
            data['prompt'],
            truncation=True,
            max_length=self.max_length,
            padding='max_length',
            return_tensors='pt',
        )

        # Encode the target answer
        target_encoding = self.tokenizer(
            data['answer'],
            truncation=True,
            max_length=self.max_length,
            padding='max_length',
            return_tensors='pt',
        )

        # Create labels by replacing pad tokens with -100
        labels = target_encoding['input_ids']
        labels[labels == self.tokenizer.pad_token_id] = -100  # Ignore padding tokens in loss computation

        return {
            'input_ids': input_encoding['input_ids'].squeeze(),
            'attention_mask': input_encoding['attention_mask'].squeeze(),
            'labels': labels.squeeze(),
        }

train_test_split = dataset['train'].train_test_split(test_size=0.3, seed=42)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

tokenizer = RobertaTokenizer.from_pretrained("Salesforce/codet5-base")
model = T5ForConditionalGeneration.from_pretrained("Salesforce/codet5-base")
# Process the training and testing data
processed_train_data = create_and_output_test_data(train_dataset)
processed_test_data = create_and_output_test_data(test_dataset)
# Process the training and testing data
train_data = CodeKGC_Dataset(processed_train_data[:], tokenizer)
test_data = CodeKGC_Dataset(processed_test_data[0:20], tokenizer)

In [ ]:
print(train_data[0])

{'input_ids': tensor([    1,  8496,  2608,   326,  5081,   637,    17,   361,  2476,  5426,
        22388,   628,   326,  3751,   977,    30,   203,   203,  9422,  5354,
           30,  2974,  2605,   326,  4183,   999,   434, 24071,   280,   477,
         1077,   330,   316,   279, 18608,   598,   324,   266,   689,   848,
         2750, 10354,  9323,   524,   749,    75,   335, 28389,   358,  3474,
          705,   305,   378,   558,    18,   203,   203,  4976,   329, 22388,
           30,     2,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0, 

In [ ]:
def finetuning_Lora(train_data, test_data, model, tokenizer, epochs=3, batch_size=1, learning_rate=5e-4,
                    max_length=256, num_beams=5, temperature=0.7, use_gpu=True, accumulation_steps=16):

    import torch
    from torch.utils.data import DataLoader
    from transformers import AdamW, get_linear_schedule_with_warmup
    from peft import get_peft_model, LoraConfig, TaskType
    from tqdm import tqdm

    device = torch.device('cuda' if torch.cuda.is_available() and use_gpu else 'cpu')
    model = model.to(device)

    # 配置 LoRA
    peft_config = LoraConfig(
        task_type=TaskType.SEQ_2_SEQ_LM,
        inference_mode=False,
        r=16,                              # LoRA 矩阵的秩
        lora_alpha=32,
        lora_dropout=0.1,
    )

    # 使用 LoRA 包装模型
    model = get_peft_model(model, peft_config)
    model.print_trainable_parameters()

    # 创建 DataLoader
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_data, batch_size=1, shuffle=False)

    # 定义优化器和学习率调度器
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    total_steps = len(train_loader) // accumulation_steps * epochs
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=int(0.1 * total_steps),
        num_training_steps=total_steps
    )

    # 训练循环
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        optimizer.zero_grad()
        for i, batch in enumerate(tqdm(train_loader, desc=f"Training Epoch {epoch+1}")):
            outputs = model(
                input_ids=batch['input_ids'].to(device),
                attention_mask=batch['attention_mask'].to(device),
                labels=batch['labels'].to(device)
            )
            loss = outputs.loss
            loss = loss / accumulation_steps  # 平均损失
            loss.backward()
            total_loss += loss.item()
            if (i + 1) % accumulation_steps == 0 or (i + 1) == len(train_loader):
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()
        avg_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch+1}/{epochs}, Average Training Loss: {avg_loss:.4f}")

        # 在每个 epoch 后进行评估
        model.eval()
        total_drug_accuracy, total_effect_accuracy = 0, 0
        num_samples = len(test_loader)

        with torch.no_grad():
            for batch in tqdm(test_loader, desc="Evaluating"):
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)

                outputs = model.generate(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    max_length=max_length,
                    num_beams=num_beams,
                    temperature=temperature,
                    early_stopping=True
                )
                generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
                print(generated_text)
                # 提取真实答案
                true_answer_ids = labels[labels != -100]
                true_answer = tokenizer.decode(true_answer_ids, skip_special_tokens=True)

                # 解析生成的和真实的三元组
                generated_triples = parse_generated_code(generated_text)
                true_triples = parse_generated_code(true_answer)

                # 转为小写进行比较
                generated_triples = [(drug.lower(), relation.lower(), effect.lower()) for drug, relation, effect in generated_triples]
                true_triples = [(drug.lower(), relation.lower(), effect.lower()) for drug, relation, effect in true_triples]

                # 评估实体准确率
                evaluation_metrics = evaluate_entity_accuracy(generated_triples, true_triples)
                total_drug_accuracy += evaluation_metrics['drug_accuracy']
                total_effect_accuracy += evaluation_metrics['effect_accuracy']

            avg_drug_accuracy = total_drug_accuracy / num_samples
            avg_effect_accuracy = total_effect_accuracy / num_samples

            print(f"Evaluation after epoch {epoch+1}:")
            print(f"Average Drug Accuracy: {avg_drug_accuracy:.4f}")
            print(f"Average Effect Accuracy: {avg_effect_accuracy:.4f}")

    # 保存微调后的模型和 LoRA 参数
    model.save_pretrained('fine_tuned_model')

    return model


In [ ]:
# Fine-tune the model
# Fine-tune the model with gradient accumulation
lora_model = finetuning_Lora(
    train_data=train_data,
    test_data=test_data,
    model=model,
    tokenizer=tokenizer,
    epochs=3,             # Adjust the number of epochs as needed
    batch_size=1,         # Reduced batch size
    learning_rate=5e-4,   # Adjusted learning rate for LoRA
    max_length=256,       # Reduced sequence length
    num_beams=5,
    temperature=0.7,
    use_gpu=True,
    accumulation_steps=16  # Set the number of accumulation steps
)

model.save_pretrained('fine_tuned_model')
tokenizer.save_pretrained('fine_tuned_model')

In [ ]:
model.save_pretrained('fine_tuned_model')
tokenizer.save_pretrained('fine_tuned_model')

In [ ]:
def test_model(test_data, model, tokenizer, max_length=512, num_beams=5, temperature=0.7, use_gpu=True):
    """
    Test the model on the provided test dataset and output accuracy.

    Args:
    - test_data (Dataset): Test dataset (instance of CodeKGC_Dataset).
    - model: The fine-tuned model.
    - tokenizer: The tokenizer to use.
    - max_length (int): Maximum sequence length.
    - num_beams (int): Number of beams for beam search during evaluation.
    - temperature (float): Temperature parameter for text generation.
    - use_gpu (bool): Whether to use GPU for evaluation.

    Returns:
    - None (prints the accuracy results).
    """
    import torch
    from torch.utils.data import DataLoader
    from tqdm import tqdm

    device = torch.device('cuda' if torch.cuda.is_available() and use_gpu else 'cpu')
    model = model.to(device)
    model.eval()

    test_loader = DataLoader(test_data, batch_size=1, shuffle=False)

    total_drug_accuracy, total_effect_accuracy = 0, 0
    num_samples = len(test_loader)

    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Testing"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_length=max_length,
                num_beams=num_beams,
                temperature=temperature,
                early_stopping=True
            )
            generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
            print(generated_text)
            # Extract the true answer from the labels
            true_answer_ids = labels[labels != -100]
            true_answer = tokenizer.decode(true_answer_ids, skip_special_tokens=True)

            # Parse generated and true triples
            generated_triples = parse_generated_code(generated_text)
            true_triples = parse_generated_code(true_answer)

            # Lowercase for comparison
            generated_triples = [(drug.lower(), relation.lower(), effect.lower()) for drug, relation, effect in generated_triples]
            true_triples = [(drug.lower(), relation.lower(), effect.lower()) for drug, relation, effect in true_triples]

            # Evaluate entity accuracy
            evaluation_metrics = evaluate_entity_accuracy(generated_triples, true_triples)
            total_drug_accuracy += evaluation_metrics['drug_accuracy']
            total_effect_accuracy += evaluation_metrics['effect_accuracy']

    avg_drug_accuracy = total_drug_accuracy / num_samples
    avg_effect_accuracy = total_effect_accuracy / num_samples

    print(f"Test Results:")
    print(f"Average Drug Accuracy: {avg_drug_accuracy:.4f}")
    print(f"Average Effect Accuracy: {avg_effect_accuracy:.4f}")


In [ ]:
# 加载保存的模型和分词器
from transformers import T5ForConditionalGeneration, T5Tokenizer
from peft import PeftModel

# 加载模型和分词器
tokenizer = RobertaTokenizer.from_pretrained('fine_tuned_model')

base_model = T5ForConditionalGeneration.from_pretrained('Salesforce/codet5-base')

# 加载 LoRA 参数
model = PeftModel.from_pretrained(base_model, 'fine_tuned_model')


# 创建测试数据集（使用您已有的 processed_test_data）
test_data = CodeKGC_Dataset(processed_test_data[0:1], tokenizer)

# 调用测试函数
test_model(
    test_data=test_data,
    model=model,
    tokenizer=tokenizer,
    max_length=512,
    num_beams=5,
    temperature=0.7,
    use_gpu=True
)


Testing:   0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Testing: 100%|██████████| 1/1 [00:01<00:00,  1.11s/it]

Extract([Triple(drug('Lp-TAE'), Cause('Adverse Effect'), adverse_effect('liver dysfunction'))])
Matched Triples: [('Lp-TAE', 'Adverse Effect', 'liver dysfunction')]
Matched Triples: [('Lp-TAE', 'Adverse Effect', 'myelo-suppression')]
Test Results:
Average Drug Accuracy: 1.0000
Average Effect Accuracy: 0.0000


## 1.2 Train and test with codekgc

In [ ]:
def prepare_input_data(text, answer):
    """
    Prepare input data with schema-aware prompt.

    Args:
    - text (str): The input text.
    - answer (str): The expected output (true label).

    Returns:
    - dict: A dictionary containing 'prompt' and 'answer'.
    """
    # Define the schema
    schema_prompt = """Schema:

Extract([Triple(drug(string), Cause(string), adverse_effect(string))])

Examples:

Text: "The patient experienced nausea after taking aspirin."
Output: Extract([Triple(drug("aspirin"), Cause("adverse effect"), adverse_effect("nausea"))])

Text: "Ibuprofen can cause dizziness in some patients."
Output: Extract([Triple(drug("Ibuprofen"), Cause("adverse effect"), adverse_effect("dizziness"))])
"""

    # Combine the schema and the input text
    prompt = f"""{schema_prompt}

Now, please extract the drug-adverse effect triples from the following text:

Text: "{text}"
Output:"""

    return {
        'prompt': prompt,
        'answer': answer.strip()  # Remove leading/trailing whitespace
    }
def create_and_output_test_data(dataset, num_samples=None):
    num_samples = num_samples if num_samples is not None else len(dataset)
    processed_data = []

    for i in range(num_samples):
        sample = dataset[i]
        text = sample['text']
        drug = sample['drug']
        effect = sample['effect']
        answer = f' Extract([Triple(drug("{drug}"), Cause("adverse effect"), adverse_effect("{effect}"))])'

        # Prepare input data with the new function
        input_data = prepare_input_data(text, answer)
        processed_data.append(input_data)

    return processed_data



In [ ]:
tokenizer = RobertaTokenizer.from_pretrained("Salesforce/codet5-base")
model = T5ForConditionalGeneration.from_pretrained("Salesforce/codet5-base")
train_test_split = dataset['train'].train_test_split(test_size=0.3, seed=42)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']
# Process the training and testing data
processed_train_data = create_and_output_test_data(train_dataset)
processed_test_data = create_and_output_test_data(test_dataset)

# Create dataset instances
train_data = CodeKGC_Dataset(processed_train_data[:], tokenizer)
test_data = CodeKGC_Dataset(processed_test_data[:100], tokenizer)
print(train_data[0])

{'input_ids': tensor([    1,  3078,    30,   203,   203,  4976,  3816, 28166,    12,  3069,
          637,    12,  1080,  3631, 21389,    12,  1080,  3631,  1261,  2476,
           67, 13867,    12,  1080,  3719,  5717,   203,   203, 23905,    30,
          203,   203,  1528,    30,   315,  1986, 18608, 31207,    72,   290,
         1579,    69,  1839, 13763, 30869,   481,   267,  1199,   203,  1447,
           30,  8152,  3816, 28166,    12,  3069,   637,  2932, 28264,   481,
          267,     6,  3631, 21389,  2932,   361,  2476,  5426,     6,  3631,
         1261,  2476,    67, 13867,  2932,    82,  1579,    69,     6,  3719,
         5717,   203,   203,  1528,    30,   315,    45,    70,   416,   303,
           74,   275,   848,  4620,   302, 25695,  6336,   316,  2690,  9670,
         5828,  1199,   203,  1447,    30,  8152,  3816, 28166,    12,  3069,
          637,  2932,    45,    70,   416,   303,    74,   275,     6,  3631,
        21389,  2932,   361,  2476,  5426,     6, 

In [ ]:
# Fine-tune the model
lora_model = finetuning_Lora(
    train_data=train_data,
    test_data=test_data,
    model=model,
    tokenizer=tokenizer,
    epochs=3,             # Adjust the number of epochs as needed
    batch_size=1,         # Adjust batch size based on your GPU memory
    learning_rate=5e-4,
    max_length=256,
    num_beams=5,
    temperature=0.7,
    use_gpu=True,
    accumulation_steps=16  # Use gradient accumulation if needed
)


trainable params: 1,769,472 || all params: 224,651,520 || trainable%: 0.7877


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Training Epoch 1: 100%|██████████| 4774/4774 [10:18<00:00,  7.72it/s]


Epoch 1/3, Average Training Loss: 0.0268


Evaluating:   0%|          | 0/100 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Evaluating:   1%|          | 1/100 [00:01<02:15,  1.37s/it]

Extract([Triple(drug("Lp-TAE"), Cause("adverse effect"), adverse_effect("liver dysfunction"))])
Matched Triples: [('Lp-TAE', 'adverse effect', 'liver dysfunction')]
Matched Triples: [('Lp-TAE', 'adverse effect', 'myelo-suppression')]


...

Extract([Triple(drug("metronidazole"), Cause("adverse effect"), adverse_effect("myopia"))])
Matched Triples: [('metronidazole', 'adverse effect', 'myopia')]
Matched Triples: [('sulphonamide', 'adverse effect', 'myopia')]


Evaluating:  99%|█████████▉| 99/100 [01:42<00:01,  1.03s/it]

Extract([Triple(drug("halcinonide"), Cause("adverse effect"), adverse_effect("glycosuria"))])
Matched Triples: [('halcinonide', 'adverse effect', 'glycosuria')]
Matched Triples: [('halcinonide', 'adverse effect', 'glycosuria')]


Evaluating: 100%|██████████| 100/100 [01:43<00:00,  1.04s/it]

Extract([Triple(drug("diuretic-propranolol"), Cause("adverse effect"), adverse_effect("Hyperglycemia"))])
Matched Triples: [('diuretic-propranolol', 'adverse effect', 'Hyperglycemia')]
Matched Triples: [('propranolol', 'adverse effect', 'Hyperglycemia')]
Evaluation after epoch 3:
Average Drug Accuracy: 0.7700
Average Effect Accuracy: 0.4800


In [ ]:
model.save_pretrained('fine_tuned_model')
tokenizer.save_pretrained('fine_tuned_model')

('fine_tuned_model/tokenizer_config.json',
 'fine_tuned_model/special_tokens_map.json',
 'fine_tuned_model/vocab.json',
 'fine_tuned_model/merges.txt',
 'fine_tuned_model/added_tokens.json')

In [ ]:
test_data = CodeKGC_Dataset(processed_test_data[:], tokenizer)

# 调用测试函数
test_model(
    test_data=test_data,
    model=model,
    tokenizer=tokenizer,
    max_length=512,
    num_beams=5,
    temperature=0.7,
    use_gpu=True
)


Testing:   0%|          | 0/2047 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Testing:   0%|          | 1/2047 [00:00<31:44,  1.07it/s]

Extract([Triple(drug("Lp-TAE"), Cause("adverse effect"), adverse_effect("liver dysfunction"))])
Matched Triples: [('Lp-TAE', 'adverse effect', 'liver dysfunction')]
Matched Triples: [('Lp-TAE', 'adverse effect', 'myelo-suppression')]


Testing:   0%|          | 2/2047 [00:01<32:11,  1.06it/s]

Extract([Triple(drug("pentamidine"), Cause("adverse effect"), adverse_effect("cardiotoxicity"))])
Matched Triples: [('pentamidine', 'adverse effect', 'cardiotoxicity')]
Matched Triples: [('pentamidine', 'adverse effect', 'second-degree heart block')]


Testing:   0%|          | 3/2047 [00:02<31:27,  1.08it/s]

...

Extract([Triple(drug("warfarin"), Cause("adverse effect"), adverse_effect("lagophthalmos"))])
Matched Triples: [('warfarin', 'adverse effect', 'lagophthalmos')]
Matched Triples: [('warfarin', 'adverse effect', 'haematoma')]


Testing: 100%|█████████▉| 2046/2047 [35:13<00:01,  1.12s/it]

Extract([Triple(drug("pentazocine"), Cause("adverse effect"), adverse_effect("gradual stiffness"))])
Matched Triples: [('pentazocine', 'adverse effect', 'gradual stiffness')]
Matched Triples: [('pentazocine', 'adverse effect', 'weakness of the lower extremities')]


Testing: 100%|██████████| 2047/2047 [35:14<00:00,  1.03s/it]

Extract([Triple(drug("prednisone"), Cause("adverse effect"), adverse_effect("pruritic bullous eruption"))])
Matched Triples: [('prednisone', 'adverse effect', 'pruritic bullous eruption')]
Matched Triples: [('prednisone', 'adverse effect', 'pruritic bullous eruption')]
Test Results:
Average Drug Accuracy: 0.7596
Average Effect Accuracy: 0.5076


## 3.use another dataset to test generalization ability
### preprocessing bc5cdr dataset

In [ ]:
!pip install bioc
!pip install datasets

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=75e3b2a17df11fd834beb56e318803691a8c727d29bad5f5e922fcf92a58a495
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26097 sha256=1bef0eec1a28fb11f3a39cf20ce067a83cd5149e9ddb1721e4f3549da3ca84d5
  Stored in directory: /root/.cache/pip/wheels/fa/80/8c/43488a924a046b733b64de3fac99252674c892a4c3801c0a61
Successfully built docopt intervaltree


In [ ]:
from datasets import load_dataset
import torch
from torch.utils.data import DataLoader
from transformers import AdamW, get_linear_schedule_with_warmup
from peft import get_peft_model, LoraConfig, TaskType
from tqdm import tqdm
from transformers import T5ForConditionalGeneration, RobertaTokenizer

dataset = load_dataset('bigbio/bc5cdr', 'bc5cdr_bigbio_kb', trust_remote_code=True)




README.md:   0%|          | 0.00/1.68k [00:00<?, ?B/s]

bc5cdr.py:   0%|          | 0.00/13.9k [00:00<?, ?B/s]

bigbiohub.py:   0%|          | 0.00/19.3k [00:00<?, ?B/s]

CDR_Data.zip:   0%|          | 0.00/20.4M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'document_id', 'passages', 'entities', 'events', 'coreferences', 'relations'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'document_id', 'passages', 'entities', 'events', 'coreferences', 'relations'],
        num_rows: 500
    })
    validation: Dataset({
        features: ['id', 'document_id', 'passages', 'entities', 'events', 'coreferences', 'relations'],
        num_rows: 500
    })
})


In [ ]:
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

import re


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
def process_bc5cdr_sample_per_sentence(sample):
    # 重建完整文本，并记录每个段落的起始位置
    passages = sample['passages']
    text = ''
    passage_id_to_offset = {}
    current_offset = 0
    for passage in passages:
        passage_text = passage['text'][0]
        passage_id_to_offset[passage['id']] = current_offset
        text += passage_text + ' '
        current_offset += len(passage_text) + 1  # 加1是因为添加了一个空格
    text = text.strip()

    # 使用句子分割将文本拆分为句子
    sentences = sent_tokenize(text)

    # 构建每个句子的起始和结束偏移量
    sentence_spans = []
    current_pos = 0
    for sentence in sentences:
        start = text.find(sentence, current_pos)
        end = start + len(sentence)
        sentence_spans.append({'text': sentence, 'start': start, 'end': end})
        current_pos = end

    # 建立实体ID到实体信息的映射，并调整偏移量
    entity_dict = {}
    for entity in sample['entities']:
        entity_offset = entity['offsets'][0]
        adjusted = False
        # 找到实体所在的句子
        for i, sentence in enumerate(sentence_spans):
            sent_start = sentence['start']
            sent_end = sentence['end']
            if entity_offset[0] >= sent_start and entity_offset[1] <= sent_end:
                entity['sentence_id'] = i  # 记录实体所在的句子ID
                adjusted = True
                break
        if adjusted:
            entity_dict[entity['id']] = entity
        else:
            continue  # 跳过不在任何句子的实体

    # 按句子处理关系
    samples = []
    for relation in sample['relations']:
        if relation['type'] == 'CID':
            arg1 = entity_dict.get(relation['arg1_id'])
            arg2 = entity_dict.get(relation['arg2_id'])
            if arg1 and arg2:
                # 确认两个实体在同一句子中
                if arg1['sentence_id'] == arg2['sentence_id']:
                    sentence = sentence_spans[arg1['sentence_id']]['text']
                    # 获取实体信息
                    if arg1['type'] == 'Chemical' and arg2['type'] == 'Disease':
                        drug_entity = arg1
                        effect_entity = arg2
                    elif arg1['type'] == 'Disease' and arg2['type'] == 'Chemical':
                        drug_entity = arg2
                        effect_entity = arg1
                    else:
                        continue  # 实体类型不匹配，跳过

                    drug_text = drug_entity['text'][0]
                    effect_text = effect_entity['text'][0]

                    # 构建模型输入的 'prompt' 和 'answer'
                    prompt = f"""Schema:

Extract([Triple(drug(string), Cause(string), adverse_effect(string))])

Examples:
Text: "The hypotensive effect of 100 mg/kg alpha-methyldopa was also partially reversed by naloxone."
Output: Extract([Triple(drug("alpha-methyldopa"), Cause("induces"), adverse_effect("hypotensive"))])
Text: "Lidocaine-induced cardiac asystole."
Output: Extract([Triple(drug("Lidocaine"), Cause("induces"), adverse_effect("cardiac asystole"))])
Sentence: "{sentence}"
Output:"""

                    answer = f'Extract([Triple(drug("{drug_text}"), Cause("induces"), adverse_effect("{effect_text}"))])'

                    sample_data = {
                        'prompt': prompt,
                        'answer': answer
                    }
                    samples.append(sample_data)
    return samples
class CodeKGC_Dataset(torch.utils.data.Dataset):
    def __init__(self, data_list, tokenizer, max_length=256):
        """
        Initializes the dataset.

        Args:
        - data_list (list): List of dictionaries with 'prompt' and 'answer'.
        - tokenizer: The tokenizer to use.
        - max_length (int): Maximum sequence length.
        """
        self.data_list = data_list
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        """
        Returns the number of samples in the dataset.
        """
        return len(self.data_list)

    def __getitem__(self, idx):
        """
        Returns a sample at the given index.

        Args:
        - idx (int): Index of the sample.

        Returns:
        - dict: A dictionary containing 'input_ids', 'attention_mask', and 'labels'.
        """
        data = self.data_list[idx]

        # Encode the input prompt
        input_encoding = self.tokenizer(
            data['prompt'],
            truncation=True,
            max_length=self.max_length,
            padding='max_length',
            return_tensors='pt',
        )

        # Encode the target answer
        target_encoding = self.tokenizer(
            data['answer'],
            truncation=True,
            max_length=self.max_length,
            padding='max_length',
            return_tensors='pt',
        )

        # Create labels by replacing pad tokens with -100
        labels = target_encoding['input_ids']
        labels[labels == self.tokenizer.pad_token_id] = -100  # Ignore padding tokens in loss computation

        return {
            'input_ids': input_encoding['input_ids'].squeeze(),
            'attention_mask': input_encoding['attention_mask'].squeeze(),
            'labels': labels.squeeze(),
        }

In [ ]:
processed_data = []
i=0
for split in ['train', 'validation', 'test']:
    for sample in dataset[split]:
        if(i<2):
          i+=1
          continue
        samples = process_bc5cdr_sample_per_sentence(sample)
        processed_data.extend(samples)
processed_data[0]

{'prompt': 'Schema:\n\nExtract([Triple(drug(string), Cause(string), adverse_effect(string))])\n\nExamples:\nText: "The hypotensive effect of 100 mg/kg alpha-methyldopa was also partially reversed by naloxone."\nOutput: Extract([Triple(drug("alpha-methyldopa"), Cause("induces"), adverse_effect("hypotensive"))])\nText: "Lidocaine-induced cardiac asystole."\nOutput: Extract([Triple(drug("Lidocaine"), Cause("induces"), adverse_effect("cardiac asystole"))])\nSentence: "Suxamethonium infusion rate and observed fasciculations."\nOutput:',
 'answer': 'Extract([Triple(drug("Suxamethonium"), Cause("induces"), adverse_effect("fasciculations"))])'}

In [ ]:
class CodeKGC_Dataset(torch.utils.data.Dataset):
    def __init__(self, data_list, tokenizer, max_length=256):
        """
        Initializes the dataset.

        Args:
        - data_list (list): List of dictionaries with 'prompt' and 'answer'.
        - tokenizer: The tokenizer to use.
        - max_length (int): Maximum sequence length.
        """
        self.data_list = data_list
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        """
        Returns the number of samples in the dataset.
        """
        return len(self.data_list)

    def __getitem__(self, idx):
        """
        Returns a sample at the given index.

        Args:
        - idx (int): Index of the sample.

        Returns:
        - dict: A dictionary containing 'input_ids', 'attention_mask', and 'labels'.
        """
        data = self.data_list[idx]

        # Encode the input prompt
        input_encoding = self.tokenizer(
            data['prompt'],
            truncation=True,
            max_length=self.max_length,
            padding='max_length',
            return_tensors='pt',
        )

        # Encode the target answer
        target_encoding = self.tokenizer(
            data['answer'],
            truncation=True,
            max_length=self.max_length,
            padding='max_length',
            return_tensors='pt',
        )

        # Create labels by replacing pad tokens with -100
        labels = target_encoding['input_ids']
        labels[labels == self.tokenizer.pad_token_id] = -100  # Ignore padding tokens in loss computation

        return {
            'input_ids': input_encoding['input_ids'].squeeze(),
            'attention_mask': input_encoding['attention_mask'].squeeze(),
            'labels': labels.squeeze(),
        }
def test_model(test_data, model, tokenizer, max_length=512, num_beams=5, temperature=0.7, use_gpu=True):
    """
    Test the model on the provided test dataset and output accuracy.

    Args:
    - test_data (Dataset): Test dataset (instance of CodeKGC_Dataset).
    - model: The fine-tuned model.
    - tokenizer: The tokenizer to use.
    - max_length (int): Maximum sequence length.
    - num_beams (int): Number of beams for beam search during evaluation.
    - temperature (float): Temperature parameter for text generation.
    - use_gpu (bool): Whether to use GPU for evaluation.

    Returns:
    - None (prints the accuracy results).
    """
    import torch
    from torch.utils.data import DataLoader
    from tqdm import tqdm

    device = torch.device('cuda' if torch.cuda.is_available() and use_gpu else 'cpu')
    model = model.to(device)
    model.eval()

    test_loader = DataLoader(test_data, batch_size=1, shuffle=False)

    total_drug_accuracy, total_effect_accuracy = 0, 0
    num_samples = len(test_loader)

    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Testing"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_length=max_length,
                num_beams=num_beams,
                temperature=temperature,
                early_stopping=True
            )
            generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
            print(generated_text)
            # Extract the true answer from the labels
            true_answer_ids = labels[labels != -100]
            true_answer = tokenizer.decode(true_answer_ids, skip_special_tokens=True)

            # Parse generated and true triples
            generated_triples = parse_generated_code(generated_text)
            true_triples = parse_generated_code(true_answer)

            # Lowercase for comparison
            generated_triples = [(drug.lower(), relation.lower(), effect.lower()) for drug, relation, effect in generated_triples]
            true_triples = [(drug.lower(), relation.lower(), effect.lower()) for drug, relation, effect in true_triples]

            # Evaluate entity accuracy
            evaluation_metrics = evaluate_entity_accuracy(generated_triples, true_triples)
            total_drug_accuracy += evaluation_metrics['drug_accuracy']
            total_effect_accuracy += evaluation_metrics['effect_accuracy']

    avg_drug_accuracy = total_drug_accuracy / num_samples
    avg_effect_accuracy = total_effect_accuracy / num_samples

    print(f"Test Results:")
    print(f"Average Drug Accuracy: {avg_drug_accuracy:.4f}")
    print(f"Average Effect Accuracy: {avg_effect_accuracy:.4f}")


In [ ]:
from sklearn.model_selection import train_test_split
from peft import PeftModel
train_data_list, test_data_list = train_test_split(processed_data, test_size=0.3, random_state=42)
# tokenizer = RobertaTokenizer.from_pretrained("Salesforce/codet5-base")
# model = T5ForConditionalGeneration.from_pretrained("Salesforce/codet5-base")
# 创建数据集实例
tokenizer = RobertaTokenizer.from_pretrained('fine_tuned_model')
train_dataset = CodeKGC_Dataset(train_data_list, tokenizer)
test_dataset = CodeKGC_Dataset(test_data_list[:], tokenizer)


base_model = T5ForConditionalGeneration.from_pretrained('Salesforce/codet5-base')

# 加载 LoRA 参数
model = PeftModel.from_pretrained(base_model, 'fine_tuned_model')


# 创建测试数据集（使用您已有的 processed_test_data）


# 调用测试函数
test_model(
    test_data=test_dataset,
    model=model,
    tokenizer=tokenizer,
    max_length=512,
    num_beams=5,
    temperature=0.7,
    use_gpu=True
)

Testing:   0%|          | 0/1783 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Testing:   0%|          | 1/1783 [00:00<28:54,  1.03it/s]

Extract([Triple(drug("rapamycin"), Cause("induces"), adverse_effect("Sirolimus"))])
Matched Triples: [('rapamycin', 'induces', 'Sirolimus')]
Matched Triples: [('Sirolimus', 'induces', 'anemia')]


Testing:   0%|          | 2/1783 [00:01<27:49,  1.07it/s]

Extract([Triple(drug("midazolam"), Cause("induces"), adverse_effect("cardiorespiratory arrest"))])
Matched Triples: [('midazolam', 'induces', 'cardiorespiratory arrest')]
Matched Triples: [('midazolam', 'induces', 'cardiorespiratory arrest')]


Testing:   0%|          | 3/1783 [00:02<27:22,  1.08it/s]

...

Extract([Triple(drug("5-azacytidine"), Cause("induces"), adverse_effect("DNA methylation"))])
Matched Triples: [('5-azacytidine', 'induces', 'DNA methylation')]
Matched Triples: [('5-azacytidine', 'induces', 'initiation of carcinogenic process')]


Testing: 100%|█████████▉| 1782/1783 [30:43<00:00,  1.00it/s]

Extract([Triple(drug("succinylcholine"), Cause("induces"), adverse_effect("fasciculation"))])
Matched Triples: [('succinylcholine', 'induces', 'fasciculation')]
Matched Triples: [('succinylcholine', 'induces', 'fasciculation')]


Testing: 100%|██████████| 1783/1783 [30:44<00:00,  1.03s/it]

Extract([Triple(drug("PTZ"), Cause("induces"), adverse_effect("convulsions"))])
Matched Triples: [('PTZ', 'induces', 'convulsions')]
Matched Triples: [('flurothyl', 'induces', 'convulsions')]
Test Results:
Average Drug Accuracy: 0.6534
Average Effect Accuracy: 0.3971


## 4.Test On Gpt and Codex


In [ ]:
!pip install openai==0.28
import openai

# 设置 OpenAI API 密钥
openai.api_key = ''

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.52.2
    Uninstalling openai-1.52.2:
      Successfully uninstalled openai-1.52.2


In [ ]:
import openai

# 设置 OpenAI API 密钥
openai.api_key = ''


def run_baseline_with_evaluation_for_codex(processed_test_data, engine="davinci-002", max_tokens=1024, temperature=0.7):

    results = []
    total_drug_accuracy, total_effect_accuracy = 0, 0
    num_samples = 0

    for i, data in enumerate(processed_test_data[7:8]):

        input_prompt = data['prompt']
        true_label = data['true_label']

        # 使用 Codex API 生成结果
        try:
            response = openai.Completion.create(
                model=engine,
                prompt=input_prompt,
                max_tokens=max_tokens,
                temperature=temperature,
                n=1
            )

            generated_code = response.choices[0]['text'].strip()
        except Exception as e:
            print(f"Error during Codex generation: {e}")
            generated_code = ""

        # 解析生成的代码和真实标签

        #print(generated_code)
        generated_triples = parse_generated_code(generated_code)
        true_triples = parse_generated_code(true_label)
        num_samples+=len(true_triples)
        generated_triples = [(drug.lower(), relation.lower(), effect.lower()) for drug, relation, effect in generated_triples]
        true_triples = [(drug.lower(), relation.lower(), effect.lower()) for drug, relation, effect in true_triples]
        # 评估生成的三元组
        evaluation_metrics = evaluate_entity_accuracy(generated_triples, true_triples)

        # 累积计算 drug_accuracy, effect_accuracy
        total_drug_accuracy += evaluation_metrics['drug_accuracy']
        total_effect_accuracy += evaluation_metrics['effect_accuracy']

        # 打印结果
        print(f"Inputed text: {data['text']}\n")
        print(f"Sample {i+1} - Generated Code:\n{generated_code}\n")
        print(f"Sample {i+1} - Generated triples:\n{generated_triples}\n")
        print(f"Sample {i+1} - true_triples:\n{true_triples}\n")
        print(f"Sample {i+1} - True Label:\n{true_label}\n")
        print(f"Evaluation Metrics: {evaluation_metrics}\n")

        results.append({
            'generated_code': generated_code,
            'true_label': true_label,
            'evaluation_metrics': evaluation_metrics
        })

    # 计算平均 drug_accuracy 和 effect_accuracy
    avg_drug_accuracy = total_drug_accuracy / num_samples
    avg_effect_accuracy = total_effect_accuracy / num_samples

    # 打印最终的平均评估结果
    print(f"Average Drug Accuracy: {avg_drug_accuracy:.4f}")
    print(f"Average Effect Accuracy: {avg_effect_accuracy:.4f}")

    # 返回结果
    return {
        'results': results,
        'average_metrics': {
            'drug_accuracy': avg_drug_accuracy,
            'effect_accuracy': avg_effect_accuracy
        }
    }


# 调用 run_baseline_with_evaluation_for_codex 函数
codex_baseline_results = run_baseline_with_evaluation_for_codex(processed_test_data)



#解析正则表达式(正则表达式是用来适应不同类型的数据，可以在Python中用re模块来进行实现):
def extract_re(name):
    pattern = re.compile(r'(?P <Name>[^ ]+)')
    text = name
    match = pattern.search(text)
    if match:
        return match.group('Name')
    return None
    #
    # 1. 根据正则表达式匹配词组:
    #2. 返回匹配词组后的第一个组:
    #3. 返回匹配词组后的第一个组的值:
    #4. 从匹配词组的第一个组中返回匹配词组的内容:

#解析正则表达式:
extract_1 = Extract([Triple(drug('gemcitabine'), Cause('Adverse Effect'), adverse_effect('renal failure')), Triple(drug('sulfamethoxazole'), Cause('Adverse Effect'), adverse_effect('itching on the trunk'))])

#获取正则表达式的匹配词组:
print(extract_1.triples[0].head, extract_1.triples[0].relation, extract_1.triples[0].tail, extract_1.relation, extract_1.tail, extract_1.triples[1].head, extract_1.triples[1].relation, extract_1.triples[1].tail, extract_1.relation, extract_1.tail)

#获取正则表达式的匹配词组:
print(extract_1.relation, extract_1.tail)

#获取正则表达式的匹配词组:
print(extract_1.triples[0].head, extract_1.triples[0].relation, extract_1.triples[0].tail, extract_1.triple

In [ ]:


def evaluate_gpt(prompt, max_tokens=150, temperature=0.7, model="gpt-4o-mini"):
    """
    使用 OpenAI GPT-3.5 Turbo 模型进行文本生成。

    Args:
    - prompt (str): 提供给 GPT 的输入提示文本。
    - max_tokens (int): GPT 生成的最大 token 数量。
    - temperature (float): 控制生成多样性的参数。
    - model (str): 要使用的模型名称，默认为 "gpt-3.5-turbo"。

    Returns:
    - str: GPT 生成的文本。
    """

    try:
        response = openai.ChatCompletion.create(
            model=model,  # 使用 gpt-3.5-turbo 或其他 chat 模型
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=max_tokens,
            temperature=temperature
        )
        # 返回 GPT 生成的文本
        return response['choices'][0]['message']['content'].strip()

    except Exception as e:
        print(f"Error: {e}")
        return None

# 测试 GPT-3.5 Turbo 的文本生成
prompt = """
Write a Python function that calculates the sum of two numbers and returns the result.
"""
generated_text = evaluate_gpt(prompt)

print("Generated Text by GPT-3.5 Turbo:\n", generated_text)



Generated Text by GPT-3.5 Turbo:
 Certainly! Here is a simple Python function that calculates the sum of two numbers and returns the result:

```python
def sum_two_numbers(a, b):
    """Returns the sum of two numbers."""
    return a + b

# Example usage:
result = sum_two_numbers(5, 3)
print("The sum is:", result)  # Output: The sum is: 8
```

You can call the `sum_two_numbers` function with any two numbers as arguments, and it will return their sum.


In [ ]:
def evaluate_dataset_with_triples(processed_test_data, max_tokens=1024, temperature=0.7, model="gpt-4o-mini"):
    """
    对整个数据集进行评估，并解析和比较生成的三元组与真实三元组。

    Args:
    - processed_test_data (list): 包含多个数据样本的列表，每个样本是一个包含 'prompt' 和 'true_label' 的字典。
    - max_tokens (int): GPT 生成的最大 token 数量。
    - temperature (float): 控制生成多样性的参数。
    - model (str): 使用的模型名称，默认为 "gpt-3.5-turbo"。

    Returns:
    - list: 包含每个样本生成结果和评估指标的列表。
    """
    results = []
    total_drug_accuracy = 0
    total_effect_accuracy = 0
    num_true=0
    for i, data in enumerate(processed_test_data[5:6]):
        prompt = data['prompt']
        true_label = data['true_label']

        # 调用 GPT 模型生成文本
        generated_text = evaluate_gpt(prompt, max_tokens=max_tokens, temperature=temperature, model=model)

        # 解析生成的三元组和真实三元组
        generated_triples = parse_generated_code(generated_text)
        true_triples = parse_generated_code(true_label)
        num_true+=len(true_triples)
        # 将三元组转换为小写格式进行比较
        generated_triples = [(drug.lower(), relation.lower(), effect.lower()) for drug, relation, effect in generated_triples]
        true_triples = [(drug.lower(), relation.lower(), effect.lower()) for drug, relation, effect in true_triples]

        # 评估生成的三元组与真实三元组的匹配情况
        evaluation_metrics = evaluate_entity_accuracy(generated_triples, true_triples)

        # 累积计算药物和不良反应的准确率
        total_drug_accuracy += evaluation_metrics['drug_accuracy']
        total_effect_accuracy += evaluation_metrics['effect_accuracy']

        # 将结果保存到列表中
        result = {
            'prompt': prompt,
            'true_label': true_label,
            'generated_text': generated_text,
            'generated_triples': generated_triples,
            'true_triples': true_triples,
            'evaluation_metrics': evaluation_metrics
        }
        results.append(result)

        # 打印结果
        print(f"Sample {i+1} - Prompt:\n{prompt}\n")
        print(f"Sample {i+1} - Generated Text:\n{generated_text}\n")
        print(f"Sample {i+1} - Generated Triples:\n{generated_triples}\n")
        print(f"Sample {i+1} - True Triples:\n{true_triples}\n")
        print(f"Sample {i+1} - Evaluation Metrics: {evaluation_metrics}\n")

    # 计算平均药物和不良反应准确率
    avg_drug_accuracy = total_drug_accuracy / num_true
    avg_effect_accuracy = total_effect_accuracy / num_true

    print(f"Average Drug Accuracy: {avg_drug_accuracy:.4f}")
    print(f"Average Effect Accuracy: {avg_effect_accuracy:.4f}")

    return results


# 假设 processed_test_data 是你的测试数据集
# 调用 evaluate_dataset_with_triples 函数对整个数据集进行测试
results = evaluate_dataset_with_triples(processed_test_data, max_tokens=150, temperature=0.7, model="gpt-3.5-turbo")

# 打印整个数据集的结果
for result in results:
    print("Prompt:", result['prompt'])
    print("Generated Text:", result['generated_text'])
    print("Generated Triples:", result['generated_triples'])
    print("True Triples:", result['true_triples'])
    print("Evaluation Metrics:", result['evaluation_metrics'])
    print("---------")


Matched Triples: [('Naproxen', 'Adverse Effect', 'dimorphic clinical pattern')]
Matched Triples: [('Naproxen', 'Adverse Effect', 'erythropoietic protoporphyria')]
Sample 1 - Prompt:

    from typing import List
    class Rel:
        def __init__(self, name: str):
            self.name = name

    class Cause(Rel):
        def __init__(self, name: str):
            super().__init__(name=name)

    class Entity:
        def __init__(self, name: str):
            self.name = name

    class drug(Entity):
        def __init__(self, name: str):
            super().__init__(name=name)

    class adverse_effect(Entity):
        def __init__(self, name: str):
            super().__init__(name=name)

    class Triple:
        def __init__(self, head: Entity, relation: Rel, tail: Entity):
            self.head = head
            self.relation = relation
            self.tail = tail

    class Extract:
        def __init__(self, triples: List[Triple] = []):
            self.triples = triples
   